In [37]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import re
import pickle
from  nltk.tokenize  import  word_tokenize

from spellcheck import *

In [2]:
print('Loading glove vectors...')
model = pickle.load(open('model_glove_wiki.pkl', 'rb'))
print('Done.')

Loading glove vectors...
Done.


In [3]:
# def clean_sentence(sentence):
#     "remove chars that are not letters or numbers, downcase, then remove stop words"
#     sentence = sentence.lower()
#     sentence = re.sub('[0-9]+.[0-9]+','9',sentence)
#     return sentence

In [4]:
# def clean_dataframe(data):
#     "drop nans, then apply 'clean_sentence' function to question1 and 2"
#     data = data.dropna()

#     for col in ['question1', 'question2']:
#         print 'Cleaning col ' + col
#         data[col] = data[col].apply(clean_sentence)

#     return data

In [3]:
print('Loading train data from csv..')
traindata = pd.read_csv('./input/train.csv')
traindata = traindata.dropna()
print('Done.')

Loading train data from csv..
Done.


In [75]:
# print('Cleaning train data...')
# cleaned_train_data = clean_dataframe(traindata)
# print('Data cleaning done...')

cleaned_train_data.to_csv('./train_cleaned.csv')

In [4]:
print('Loading test data from csv..')
testdata = pd.read_csv('./input/test.csv')
testdata = testdata.dropna()
print('Done.')

Loading test data from csv..
Done.


In [6]:
# print('Cleaning test data...')
# cleaned_test_data = clean_dataframe(testdata)
# print('Data cleaning done...')

# cleaned_test_data.to_csv('./input/test_cleaned.csv')

In [70]:
def reduceGloveModel(gloveFile, model, index, last_index, missed, missed_set, data):
    print "Starting Glove Model Reduction..."
    
    clean_sentences = list()
    q = last_index
    for i, sentence in enumerate(data):
        if i%10000 == 0:
            print 'i: ',i
        
        sentence = sentence.lower()
        sentence = re.sub('[0-9]+.[0-9]+',' 9 ',sentence)
        sentence = re.sub('[0-9]+',' 9 ',sentence)
        sentence = re.sub(' [a-zA-Z0-9]+.com[ .,?!]+',' website ',sentence)
        
        new_sentence = []
        for word in word_tokenize(unicode(sentence, errors='ignore').encode('ascii','ignore')):
            if word == '':
                continue
            if word in gloveFile and word in model:
                new_sentence.append(word)
                continue
            if word in gloveFile and word not in model:
                q = q+1
                index[word] = q
                model[word] = gloveFile[word]
                new_sentence.append(word)
                continue
            if word not in gloveFile:
                split_words = re.split("[+_/\\\\=-]+",word)
                for nword in split_words:
                    if nword in gloveFile and nword in model:
                        new_sentence.append(nword)
                        continue
                    if nword in gloveFile and nword not in model:
                        q = q+1
                        index[nword] = q
                        model[nword] = gloveFile[nword]
                        new_sentence.append(nword)
                        continue
                    if nword not in gloveFile:
                        cword = correction(nword)
                    if cword in gloveFile and cword in model:
                        new_sentence.append(cword)
                        continue
                    if cword in gloveFile and cword not in model:
                        q = q+1
                        index[cword] = q
                        model[cword] = gloveFile[cword]
                        new_sentence.append(cword)
                        continue
                    new_sentence.append('unk')
#                     print nword
                    missed_set.add(nword)
                    missed = len(missed_set)
                    if missed % 100 == 0:
                        print 'missed: ',missed
        # Add spellchecked and verified words to list of wentences
        clean_sentences.append(" ".join(new_sentence))
      
    print "Done. Model reduced to ",len(model)
    print "Done. Index reduced to ",len(index)
    return model,index,q,missed,clean_sentences,missed_set

In [18]:
# c1 = list(data['question1'].values)
# c2 = list(data['question2'].values)
# q_concat = c1 + c2
# new_model = {}
# for i, sentence in enumerate(q_concat):
# #     t = [word for word in sentence.split(" ") if word in model]
#     new_model.update( dict([ (word, model[word]) for word in sentence.split(" ") if word in model ]) )

In [63]:
cleaned_train_data = traindata

In [64]:
new_model = {}
new_index = {}
new_index['unk']=0
new_model['unk']=model['unk']
missed_words = 0
last_index = 0
new_model, new_index, last_index, missed_words, cleaned_train_data['question1'], missed_set = reduceGloveModel(model, new_model, 
                                                           new_index, last_index, missed_words, cleaned_train_data['question1'].values)

Starting Glove Model Reduction...
i:  0
missed:  100
missed:  100
missed:  200
i:  10000
missed:  300
missed:  400
missed:  500
missed:  500
i:  20000
missed:  600
missed:  700
missed:  700
i:  30000
missed:  800
missed:  900
missed:  1000
i:  40000
missed:  1100
missed:  1200
i:  50000
missed:  1300
missed:  1400
i:  60000
missed:  1500
missed:  1600
i:  70000
missed:  1700
missed:  1800
missed:  1800
i:  80000
missed:  1900
missed:  2000
missed:  2100
i:  90000
missed:  2200
i:  100000
missed:  2300
missed:  2300
missed:  2400
i:  110000
missed:  2500
missed:  2500
missed:  2500
missed:  2600
missed:  2600
i:  120000
missed:  2700
missed:  2800
missed:  2800
missed:  2800
i:  130000
missed:  2900
missed:  3000
missed:  3000
i:  140000
missed:  3100
missed:  3200
i:  150000
missed:  3300
missed:  3400
missed:  3400
missed:  3400
i:  160000
missed:  3500
missed:  3600
missed:  3600
missed:  3600
i:  170000
missed:  3700
missed:  3800
i:  180000
missed:  3900
missed:  3900
i:  190000
mi

In [71]:
new_model, new_index, last_index, missed_words, cleaned_train_data['question2'], missed_set = reduceGloveModel(model, new_model, 
                                                           new_index, last_index, missed_words, missed_set,cleaned_train_data['question2'].values)

Starting Glove Model Reduction...
i:  0
missed:  7400
missed:  7400
missed:  7400
missed:  7400
i:  10000
missed:  7500
i:  20000
missed:  7600
missed:  7600
missed:  7600
missed:  7600
missed:  7600
missed:  7600
missed:  7600
i:  30000
missed:  7700
missed:  7700
missed:  7700
i:  40000
missed:  7800
i:  50000
missed:  7900
missed:  7900
missed:  8000
missed:  8000
missed:  8000
i:  60000
missed:  8100
missed:  8100
i:  70000
missed:  8200
i:  80000
missed:  8300
i:  90000
missed:  8400
i:  100000
i:  110000
missed:  8500
missed:  8500
missed:  8500
i:  120000
missed:  8600
missed:  8600
missed:  8600
missed:  8600
missed:  8600
missed:  8600
missed:  8600
missed:  8600
i:  130000
missed:  8700
missed:  8700
missed:  8700
missed:  8700
missed:  8700
missed:  8700
missed:  8700
missed:  8700
i:  140000
missed:  8800
missed:  8800
missed:  8800
missed:  8800
i:  150000
missed:  8900
i:  160000
missed:  9000
missed:  9000
missed:  9000
missed:  9000
missed:  9000
i:  170000
missed:  910

In [72]:
print 'Original word bank: ',len(model)
print 'Reduced word bank: ',len(new_model)
print 'Percent reduction: ',float(len(model)-len(new_model))*100/len(model)
print 'Missed words: ',missed_words
print last_index

Original word bank:  400000
Reduced word bank:  62524
Percent reduction:  84.369
Missed words:  11231
62523


In [73]:
print('Saving model...')
with open('reduce_glove_wiki.pkl', 'wb') as output:
    pickle.dump(new_model, output, pickle.HIGHEST_PROTOCOL)
print('Done.')

Saving model...
Done.


In [74]:
print('Saving indices...')
with open('reduce_glove_wiki_index.pkl', 'wb') as output:
    pickle.dump(new_index, output, pickle.HIGHEST_PROTOCOL)
print('Done.')

Saving indices...
Done.


In [3]:
print('Loading test data from csv..')
cleaned_train_data = pd.read_csv('./input/train_cleaned.csv')
print('Done.')

Loading test data from csv..
Done.


In [6]:
c1 = list(cleaned_train_data['question1'].values)
c2 = list(cleaned_train_data['question2'].values)
q_concat = c1 + c2
c=set()
for i, sentence in enumerate(q_concat):
    for word in word_tokenize(unicode(sentence, errors='ignore').encode('ascii','ignore')):
        if word not in model and word != '':
            c.add(word)
            if i%10000 == 0:
                print i
            
print len(c)
print c
# f = set(e)
# print len(f)
# print float(len(d))/len(f)*100

150000
150000
180000
210000
280000
340000
370000
440000
510000
510000
510000
510000
510000
620000
620000
43918
set(['yeo/no', 'pieces/rest', 'isomorphus', 'mc/period', 'daiict', 'mothers/women', 'devonians', '^9z/', '99^99^99^99^n', 'gaban/godan', 'befikre', 'paiza', 'do/while', 'lewyn', 'chaganti', 'tdv9', 'restroom/fitting', '9999gbp', 'post-gate', 'funforlouis', 'shadowdancer', 'vasuda', 'emkit', '/sum', 'q-q', 'paraphasia', 'number-9', 'jayakanthan', 'gazatted', '99-key', 'entry/landing/conversion', 'throwaround', 'oxidized/damaged', 'books/novels', 'rabbitmq', 'tambin', 'sericin', 'marsden/tromba', '/projects', 'girokonto', 'legalforce/trademarkia', 'x^9-9px+9p^9', 'youth9work.com', 'punchable', '-the-', "'about", '9.9=p/q', 'ffor', 'micro-surveys', 'channel999.com', '9*99^-99', 'washerman', 'j99', 'a9-b9', 'white-out', 'capital-output', 'starboy', 'prodotto', 'arrowverse', 'myersbriggs', 'krampusnacht', 'shows.which', 'alchemyapi', "'pleasure", 'naache', 'writometer', 'china/indi

In [14]:
print('Reading csv...')
data1 = pd.read_csv('./input/train_cleaned_e.csv')
data1 = data1.dropna()
print('Done...')

Reading csv...
Done...


In [86]:
len(new_index)

62524

In [62]:
model["ca"]

[0.028222,
 -0.072881,
 0.56781,
 -0.29649,
 0.08538,
 -0.78858,
 -0.20837,
 0.22496,
 -0.55122,
 0.29046,
 -0.22736,
 0.6031,
 -0.21876,
 -0.33138,
 0.92313,
 0.83643,
 0.47158,
 0.18449,
 0.35594,
 -0.83259,
 -0.51416,
 0.282,
 0.36725,
 0.74079,
 0.59935,
 -1.8163,
 -1.0256,
 -0.095992,
 0.91687,
 -1.2579,
 2.8979,
 0.68416,
 -0.78139,
 -0.18432,
 -0.14208,
 -0.474,
 0.10048,
 -0.17088,
 1.0221,
 -0.62035,
 -0.19287,
 -0.076375,
 0.42848,
 0.73572,
 -0.10176,
 0.43616,
 0.19084,
 -0.2813,
 0.14178,
 1.0011]

In [77]:
missed_set

{'umtc',
 'metrino',
 'randsfjorden',
 'intercaste',
 'careprost',
 'isomorphus',
 'autoposter',
 'throwaround',
 'saxonland',
 'haichao',
 'euthenesia',
 'life.already',
 'backtests',
 'bhagbat',
 "'irreverent",
 'chaganti',
 'brahmstra',
 'not..he',
 'crunchprep',
 'carryminati',
 'googlenet',
 'hydrosills',
 'shadowdancer',
 'brazuca',
 'schizoids',
 'pokeballs',
 'nature.how',
 'spitians',
 'chlorophyl',
 'danbilzerian',
 'paraphasia',
 'sharukh',
 'bulksfor',
 'healing.how',
 'gimmic',
 'electrovalent',
 'jayakanthan',
 'pmjdy',
 'magarpatta',
 'rhomobile',
 'a.p.c',
 'secondlife',
 'b.lafter',
 'sbcglobal.net',
 'a.p.j',
 'months.what',
 'sericin',
 'listview',
 'girokonto',
 'org.secure.smsgps',
 'punchable',
 'weeks.what',
 'youtu.be',
 'zeronet',
 'incrementer',
 'professorials',
 'fonepad',
 "'must'or'had",
 'jeevansathi',
 'eveteasing',
 'chargebee',
 'arrowverse',
 'fxkart',
 'neopet',
 'yaazhai',
 'codesprint',
 'krampusnacht',
 'unpreventable',
 'aldosteronism',
 'youbroa

In [85]:
cleaned_train_data['question2'][10]

'what are some of the things technicians can tell about the durability and reliability of laptops and its components ?'